In [10]:
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd

In [11]:
# Declare browser
options = webdriver.ChromeOptions()
chrome_options = Options()
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)
# Open URL
driver.get("https://daitheky.net/properties?keyword=ban%20dat&offset=0&size=12")
sleep(random.randint(5,10))

In [ ]:
count = 1
links, title, address = [], [], []
while count<=834:
    try:
        print("Crawl links Page " + str(count))
        # ================================ GET link
        elems = driver.find_elements(By.CSS_SELECTOR , ".title [href]")
        links += [elem.get_attribute('href') for elem in elems]

        # ================================ GET title
        elems = driver.find_elements(By.CSS_SELECTOR , ".title")
        title += [elem.text for elem in elems]
        # ================================ GET address
        elems_address = driver.find_elements(By.CSS_SELECTOR , ".address span")
        address += [elem.text for elem in elems_address]
        # ================================ Click Next Page Button
        next_page = driver.find_element("xpath", "/html/body/app-root/div/app-pages/mat-sidenav-container/mat-sidenav-content/main/app-properties/div/div/mat-sidenav-container/mat-sidenav-content/div/div[16]/mat-paginator/div/div/div[2]/button[2]")
        next_page.click()
        print("Clicked on button next page!")
        sleep(random.randint(4,6))
        count += 1
    except ElementNotInteractableException:
        print("Element Not Interactable Exception!")
        break
df1 = pd.DataFrame(list(zip(title, address, links)), columns = ['title', 'address', 'item_links'])
df1['index_']= np.arange(1, len(df1) + 1)


In [ ]:
# ================================ create csv (title, address, item_links)
df1.to_csv('LandLinks.csv',index=True)
print("done")

In [ ]:
price, typeBDS, facade, area = [], [], [], []
for i in range(len(links)):
    driver.get(links[i])
    sleep(6)
    if driver.find_elements("xpath","/html/body/app-root/div/app-pages/mat-sidenav-container/mat-sidenav-content/main/app-property/div[1]/div/mat-sidenav-container/mat-sidenav-content/mat-card[4]/div/div[3]/span[2]"):
        # ================================ GET price
        elems_price = driver.find_elements(By.CSS_SELECTOR , ".price-value")
        price += [elem.text for elem in elems_price]
        # ================================ GET type
        elems_typeBDS = driver.find_elements("xpath","/html/body/app-root/div/app-pages/mat-sidenav-container/mat-sidenav-content/main/app-property/div[1]/div/mat-sidenav-container/mat-sidenav-content/mat-card[4]/div/div[3]/span[2]")
        typeBDS += [elem.text for elem in elems_typeBDS]    
        # ================================ GET facade
        elems_facade = driver.find_elements("xpath","/html/body/app-root/div/app-pages/mat-sidenav-container/mat-sidenav-content/main/app-property/div[1]/div/mat-sidenav-container/mat-sidenav-content/mat-card[4]/div/div[6]/span[2]")
        facade += [elem.text for elem in elems_facade]
        # ================================ GET area
        elems_area = driver.find_elements("xpath","/html/body/app-root/div/app-pages/mat-sidenav-container/mat-sidenav-content/main/app-property/div[1]/div/mat-sidenav-container/mat-sidenav-content/mat-card[4]/div/div[8]/span[2]")
        area += [elem.text for elem in elems_area]
    else:
        price += ["not"]
        typeBDS += ["not"]
        facade += ["not"]
        area += ["not"]
# df2 = pd.DataFrame(list(zip(price, typeBDS, floor, bebroom, facade, wc, area)), columns = ['price', 'real property', 'floor', 'bebroom', 'facade', 'wc', 'area'])
# df2['detail_idx']= np.arange(1, len(df2) + 1)

In [14]:
df2 = pd.DataFrame(list(zip(price, typeBDS, facade, area)), columns = ['price', 'real property', 'facade', 'area'])
df2['detail_idx']= np.arange(1, len(df2) + 1)

In [ ]:
df2

In [16]:
df2.to_csv('LandDetail.csv',index=True)
print("done csv")

done csv


In [17]:
# ================================ link df1 vs df2
df3 = df1.merge(df2, how='left', left_on='index_', right_on='detail_idx')
# ================================ copy 3239 rows of df3 -> df4
df4 = df3.iloc[:3239].copy()
# delete rows: detail = 'not'
df4 = df4[df4['price'] != 'not']

df4 = df4[df4['real property'] != 'Bán nhà riêng']
df4 = df4[df4['real property'] != 'Bán nhà mặt phố']
df4 = df4[df4['real property'] != 'Bán nhà biệt thự, liền kề']
df4 = df4[df4['real property'] != 'Bán căn hộ chung cư']
df4 = df4[df4['real property'] != 'Bán kho, nhà xưởng']
df4 = df4[df4['real property'] != 'Cho thuê cửa hàng - ki ốt']
# ================================ Drop count index
df4 = df4.drop(columns=['index_', 'detail_idx','item_links'])

In [ ]:
df4

In [19]:
# ================================ create csv
df4.to_csv('LandData.csv',index=True)
print("done csv")
# ================================ create json
df4.to_json('LandData.json', orient='table')
print("done Json")

done csv
done Json
